In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import boto3
from numpy import array
from numpy import hstack, vstack
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import s3fs
from numpy import array
from numpy import hstack, vstack
from statsmodels.tsa.vector_ar.var_model import VAR
from random import random
import gpflow
from gpflow.utilities import print_summary
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!pip install gpflow

In [ ]:
!pip install -U tensorflow --no-cache-dir

In [ ]:
def data_files_fromS3(header_name, datafile_name, bucket_name):
    """Helper utility function that can fetch the .csv files from S3 and combine them with their column names and convert the datetime format from Unix epochs
        Arguments:
        ---------
        header_name : name of the directory your data file/column names file is in
        datafile_name: name of the file that contains the actual sensor readings ( scaled data)
        bucket_name: s3 bucket where your files live in"""
    column_names = []
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=header_name)
    header = obj['Body'].read()
    text = str(header).split(',')
    column_names = []
    for n in text:
        if n.replace('\\r\\n', '').replace("''",'') != '':
            if "b'time" in n:
                column_names.append('time')
            else:
                if "'" not in n:
                    column_names.append(n.replace('\\r\\n', ''))
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=datafile_name)
    scaled_data = pd.read_csv(obj['Body'], index_col=False, names = column_names)
    scaled_data['time'] = pd.to_datetime(scaled_data['time'],unit='s')
    scaled_data = scaled_data.set_index('time')
    return scaled_data

In [ ]:
def datapreprocessing(dataset, smooth=False, filter_nl=False):
    """Helper utility function that creates the dataset of (initial ice inventory, load, final ice inventory). This is calculated by integrating
        the instantaneous load using trapezoidal rule for each possible time interval for the entire dataset. So if the data is for 
        Parameters:
        ---------
        dataset: pandas dataframe containing the time series data
        header_name : name of the directory your data file/column names file is in
        datafile_name: name of the file that contains the actual sensor readings ( scaled data)
        bucket_name: s3 bucket where your files live in
        
        Return:
        -------
        Tuple containing (input_dataset, output_ts_meter, instantaneous load)
        """
    if filter_nl:
        dataset = dataset[(dataset['ts_meter'] > 30) & (dataset['ts_meter'] < 85)]
    ts_f = np.array(dataset['ts_f'])
    ts_in_rtd = np.array(dataset['ts_in_rtd'])
    ts_meter = np.array(dataset['ts_meter'])
    ts_out_rtd = np.array(dataset['ts_out_rtd'])
    if smooth:
        smooth_ts_f = np.where(( (ts_f <= np.median(ts_f) + 2) | (ts_f == 0)), ts_f, np.median(ts_f))
        instantaneous_load = smooth_ts_f * 0.00006309 * 1030 * (ts_in_rtd - ts_out_rtd) * (5/9) * 3.9
    else:
        instantaneous_load = ts_f * 0.00006309 * 1030 * (ts_in_rtd - ts_out_rtd) * (5/9) * 3.9
    load_data = dict()
    for i in range(0, instantaneous_load.size-2):
        for j in range(i, instantaneous_load.size-1):
            load_data[''+str(i)+':'+str(j)] = np.trapz(instantaneous_load[i:j], [x for x in range(0,instantaneous_load[i:j].size)])/6
    from_ts_meter = []
    to_ts_meter = []
    load = []
    for ft in load_data:
        split = ft.split(':')
        from_ts, to_ts = int(split[0]), int(split[1])
        from_ts_meter.append(ts_meter[from_ts])
        to_ts_meter.append(ts_meter[to_ts])
        load.append(load_data[ft])
    from_ts_meter = np.array(from_ts_meter).reshape((len(from_ts_meter), 1))
    to_ts_meter = np.array(to_ts_meter).reshape((len(to_ts_meter), 1))
    load = np.array(load).reshape((len(load), 1))
    dataset_ts_meter = np.hstack((from_ts_meter, load))
    return dataset_ts_meter, to_ts_meter, instantaneous_load

In [ ]:
def convert_to_SI(dataset):
    dataset['ts_f'] *= 3.785
    dataset['ts_in_rtd'] = (dataset['ts_in_rtd'] - 32)*(5/9)
    dataset['ts_out_rtd'] = (dataset['ts_out_rtd'] - 32)*(5/9)
    return dataset

In [ ]:
def visualize_dataset(dataset, dataset_name):
    """visualizeS ts_f, ts_in_rtd, ts_meter and ts_out_rtd features of the dataset
    in the future I need to change the function so that one can pass the features they would like to visualize and also the sample rate"""
    ice_tank_columns = dataset.filter(['ts_f', 'ts_in_rtd', 'ts_meter', 'ts_out_rtd'], axis = 1)
    ice_tank_columns = convert_to_SI(ice_tank_columns)
    #downsampled_data = ice_tank_columns.resample('10T').mean()
    downsampled_data = ice_tank_columns
    ts_f = array(downsampled_data['ts_f'])
    ts_in_rtd = array(downsampled_data['ts_in_rtd'])
    ts_meter= array(downsampled_data['ts_meter'])
    ts_out_rtd = array(downsampled_data['ts_out_rtd'])
    timestep = array([X for X in range(0, len(ts_f)*10, 10)])
    layout = go.Layout(title='Visualize dataset', xaxis={'title':'timestep'}, yaxis=dict(title='Actual values for features'), hovermode='closest')
    fig = go.Figure(layout_yaxis_range = [min(min(ts_f), min(ts_in_rtd), min(ts_meter), min(ts_out_rtd)-20), max(max(ts_f), max(ts_in_rtd), max(ts_meter), max(ts_out_rtd))+20])
    fig.add_trace(go.Scatter(x=timestep, y=ts_f, mode='lines+markers', name='ts_f (L/min)'))
    fig.add_trace(go.Scatter(x=timestep, y=ts_in_rtd, mode='lines+markers', name='ts_in_rtd (°C)'))
    fig.add_trace(go.Scatter(x=timestep, y=ts_meter, mode='lines+markers', name='ts_meter %'))
    fig.add_trace(go.Scatter(x=timestep, y=ts_out_rtd, mode='lines+markers', name='ts_out_rtd (°C)'))
    fig.update_layout(title="Feature Plot", xaxis_title="time (min)", yaxis_title="Variable Values", 
                          legend_title="State Variables", font=dict(family="Courier New, monospace", size=18, color="RebeccaPurple"))
    pyo.plot(fig, filename=dataset_name)

In [ ]:
scaled_data_2018_08_22_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
original_discharge_input4, original_discharge_output4, _ = datapreprocessing(scaled_data_2018_08_22_run2, False, True)
original_discharge_input4_smooth, original_discharge_output4_smooth, _ = datapreprocessing(scaled_data_2018_08_22_run2, True, True)

# Larger Data Models

### Charge Mode

In [ ]:
scaled_data_2018_08_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_09_run1 = scaled_data_2018_08_09_run1.resample('10T').mean()
original_charge_training_input1, original_charge_training_output1, _ = datapreprocessing(scaled_data_2018_08_09_run1, False, True)

In [ ]:
scaled_data_2018_08_15_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_15_run1 = scaled_data_2018_08_15_run1.resample('10T').mean()
original_charge_training_input2, original_charge_training_output2, _ = datapreprocessing(scaled_data_2018_08_15_run1, False, True)

In [ ]:
scaled_data_2018_08_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2018_08_22_run1 = scaled_data_2018_08_22_run1.resample('10T').mean()
original_charge_training_input3, original_charge_training_output3, _ = datapreprocessing(scaled_data_2018_08_22_run1, False, True)

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input4, original_charge_training_output4, _ = datapreprocessing(scaled_data_2019_10_08_run1, False, True)

In [ ]:
combined_input = np.vstack((original_charge_training_input1, original_charge_training_input2, original_charge_training_input3, original_charge_training_input4))
combined_to_ts_meter =  np.vstack((original_charge_training_output1, original_charge_training_output2, original_charge_training_output3, original_charge_training_output4))

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_charge_training_input1, original_charge_training_output1, _ = datapreprocessing(scaled_data_2018_05_09_run1, False, True)


scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_charge_training_input2, original_charge_training_output2, _ = datapreprocessing(scaled_data_2018_05_17_run2, False, True)


scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input3, original_charge_training_output3, _ = datapreprocessing(scaled_data_2019_10_08_run1, False, True)


scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_training_input4, original_charge_training_output4, _ = datapreprocessing(scaled_data_2019_10_22_run1, False, True)

In [ ]:
plt_mean_squared_error = []
regression_model = LinearRegression()
regression_model.fit(combined_input, combined_to_ts_meter)

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False, True)

In [ ]:
LReg_prediction_charge_mode = regression_model.predict(original_charge_validation_input)
mean_squared_error(original_charge_validation_output, LReg_prediction_charge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
#ax.scatter(original_charge_training_input1[:,1], (LReg_prediction_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_training_input1[:,1], (original_charge_training_output1.flatten() - original_charge_training_input1[:,0].flatten()),  marker='.', s=10, label='dataset1', color='b')
ax.scatter(original_charge_training_input2[:,1], (original_charge_training_output2.flatten() - original_charge_training_input2[:,0].flatten()),  marker='.', s=10, label='dataset2', color='g')
ax.scatter(original_charge_training_input3[:,1], (original_charge_training_output3.flatten() - original_charge_training_input3[:,0].flatten()),  marker='.', s=10, label='dataset3', color='r')
ax.scatter(original_charge_training_input4[:,1], (original_charge_training_output4.flatten() - original_charge_training_input4[:,0].flatten()),  marker='.', s=10, label='dataset4', color='c')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()),  marker='.', s=10, label='Test Data', color='brown')
#ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc='upper right', prop={'size': 10})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

### neural net larger data charge mode

In [ ]:
model = Sequential()
model.add(Dense(15, activation = 'relu', input_shape = (2,), kernel_initializer='glorot_uniform'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))
model.compile(optimizer='adam', loss='mse')
model.fit(combined_input, combined_to_ts_meter, epochs=300, verbose=0)

In [ ]:
NeuralNet_prediction_charge_mode = model.predict(original_charge_validation_input)
mean_squared_error(original_charge_validation_output, NeuralNet_prediction_charge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (NeuralNet_prediction_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()),  marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

### larger data discharge mode

In [ ]:
scaled_data_2018_05_09_run1
scaled_data_2018_05_14_run1
scaled_data_2018_05_17_run2
scaled_data_2018_08_25_run2

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_input1, original_discharge_output1, _ = datapreprocessing(scaled_data_2018_05_09_run1, False, True)
original_discharge_input1_smooth, original_discharge_output1_smooth, _ = datapreprocessing(scaled_data_2018_05_09_run1, True, True)

In [ ]:
scaled_data_2018_05_14_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_14_run1 = scaled_data_2018_05_14_run1.resample('10T').mean()
original_discharge_input2, original_discharge_output2, _ = datapreprocessing(scaled_data_2018_05_14_run1, False, True)
original_discharge_input2_smooth, original_discharge_output2_smooth, _ = datapreprocessing(scaled_data_2018_05_14_run1, True, True)

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_input3, original_discharge_output3, _ = datapreprocessing(scaled_data_2018_05_17_run2, False, True)
original_discharge_input3_smooth, original_discharge_output3_smooth, _ = datapreprocessing(scaled_data_2018_05_17_run2, True, True)

In [ ]:
scaled_data_2018_08_25_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_25_run2 = scaled_data_2018_08_25_run2.resample('10T').mean()
original_discharge_input4, original_discharge_output4, _ = datapreprocessing(scaled_data_2018_08_25_run2, False, True)
original_discharge_input4_smooth, original_discharge_output4_smooth, _ = datapreprocessing(scaled_data_2018_08_25_run2, True, True)

In [ ]:
scaled_data_2018_08_22_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_22_run2 = scaled_data_2018_08_22_run2.resample('10T').mean()
original_discharge_input5, original_discharge_output5, _ = datapreprocessing(scaled_data_2018_08_22_run2, False, True)
original_discharge_input5_smooth, original_discharge_output5_smooth, _ = datapreprocessing(scaled_data_2018_08_22_run2, True, True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_input1[:,1], (original_discharge_output1.flatten() - original_discharge_input1[:,0].flatten()),  marker='.', s=10, label='Test Data', color='brown')
ax.scatter(original_discharge_input2[:,1], (original_discharge_output2.flatten() - original_discharge_input2[:,0].flatten()),  marker='.', s=10, label='Dataset1', color='b')
ax.scatter(original_discharge_input3[:,1], (original_discharge_output3.flatten() - original_discharge_input3[:,0].flatten()),  marker='.', s=10, label='Dataset2', color='g')
ax.scatter(original_discharge_input4[:,1], (original_discharge_output4.flatten() - original_discharge_input4[:,0].flatten()),  marker='.', s=10, label='Dataset3', color='r')
ax.scatter(original_discharge_input5[:,1], (original_discharge_output5.flatten() - original_discharge_input5[:,0].flatten()),  marker='.', s=10, label='Dataset4', color='c')
ax.legend(loc='upper right')
ax.legend( prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
#ax.scatter(original_charge_training_input1[:,1], (LReg_prediction_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_input1_smooth[:,1], (original_discharge_output1_smooth.flatten() - original_discharge_input1_smooth[:,0].flatten()),  marker='.', s=10, label='Test Data')
ax.scatter(original_discharge_input2_smooth[:,1], (original_discharge_output2_smooth.flatten() - original_discharge_input2_smooth[:,0].flatten()),  marker='.', s=10, label='Dataset1')
ax.scatter(original_discharge_input3_smooth[:,1], (original_discharge_output3_smooth.flatten() - original_discharge_input3_smooth[:,0].flatten()),  marker='.', s=10, label='Dataset2')
ax.scatter(original_discharge_input4_smooth[:,1], (original_discharge_output4_smooth.flatten() - original_discharge_input4_smooth[:,0].flatten()),  marker='.', s=10, label='Dataset3')
ax.scatter(original_discharge_input5_smooth[:,1], (original_discharge_output5_smooth.flatten() - original_discharge_input5_smooth[:,0].flatten()),  marker='.', s=10, label='Dataset4')
#ax.scatter(original_discharge_input1[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()),  marker='.', s=10, label='test', color='red')
#ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend(loc='upper right', prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
combined_input_discharge = np.vstack((original_discharge_test_input, original_discharge_input2, original_discharge_input3, original_discharge_input4))
combined_to_ts_meter_discharge =  np.vstack((original_discharge_test_output, original_discharge_output2, original_discharge_output3, original_discharge_output4))

In [ ]:
regression_model_discharge = LinearRegression()
regression_model_discharge.fit(combined_input_discharge, combined_to_ts_meter_discharge)

In [ ]:
scaled_data_2018_08_15_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_15_run2 = scaled_data_2018_08_15_run2.resample('10T').mean()
original_discharge_test_input, original_discharge_test_output, _ = datapreprocessing(scaled_data_2018_08_15_run2, False)
original_discharge_test_input_smooth, original_discharge_test_output_smooth, _ = datapreprocessing(scaled_data_2018_08_15_run2, True)

In [ ]:
visualize_dataset(scaled_data_2018_08_15_run2, './scaled_data_2018_08_15_run2.html')

In [ ]:
LReg_prediction_discharge_mode_nonsmooth = regression_model_discharge.predict(original_discharge_input1)
mean_squared_error(original_discharge_output1, LReg_prediction_discharge_mode_nonsmooth, squared=False)

In [ ]:
combined_input_discharge_smooth = np.vstack((original_discharge_test_input_smooth, original_discharge_input2_smooth, original_discharge_input3_smooth, original_discharge_input4_smooth))
combined_to_ts_meter_discharge_smooth =  np.vstack((original_discharge_test_output_smooth, original_discharge_output2_smooth, original_discharge_output3_smooth, original_discharge_output4_smooth))

In [ ]:
regression_model_discharge_smooth = LinearRegression()
regression_model_discharge_smooth.fit(combined_input_discharge_smooth, combined_to_ts_meter_discharge_smooth)

In [ ]:
LReg_prediction_discharge_mode_smooth = regression_model_discharge_smooth.predict(original_discharge_input1_smooth)
mean_squared_error(original_discharge_output1_smooth, LReg_prediction_discharge_mode_smooth, squared=False)

In [ ]:
scaled_data_2018_08_15_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_08_15_run2 = scaled_data_2018_08_15_run2.resample('10T').mean()
visualize_dataset(scaled_data_2018_08_15_run2, './scaled_data/scaled_data_2018_08_15_run2.html')

### linear regression charge mode filtered data

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input4, original_charge_training_output4, _ = datapreprocessing(scaled_data_2019_10_08_run1, False, True)

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False, True)

In [ ]:
regression_model_charge = LinearRegression()
regression_model_charge.fit(original_charge_training_input4, original_charge_training_output4)

In [ ]:
LReg_prediction_charge_mode = regression_model_charge.predict(original_charge_validation_input)
mean_squared_error(original_charge_validation_output, LReg_prediction_charge_mode, squared=False)

### linear regression discharge mode smooth vs nonsmooth filtered data

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_input1_filter, original_discharge_output1_filter, _ = datapreprocessing(scaled_data_2018_05_09_run1, False, True )
original_discharge_input1_smooth_filter, original_discharge_output1_smooth_filter, _ = datapreprocessing(scaled_data_2018_05_09_run1, True, True)

In [ ]:
scaled_data_2018_05_14_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_14_run1 = scaled_data_2018_05_14_run1.resample('10T').mean()
original_discharge_input2_filter, original_discharge_output2_filter, _ = datapreprocessing(scaled_data_2018_05_14_run1, False, True)
original_discharge_input2_smooth_filter, original_discharge_output2_smooth_filter, _ = datapreprocessing(scaled_data_2018_05_14_run1, True, True)

In [ ]:
regression_model_discharge = LinearRegression()
regression_model_discharge.fit(original_discharge_input1_smooth_filter, original_discharge_output1_smooth_filter)

In [ ]:
LReg_prediction_charge_mode = regression_model_charge.predict(original_discharge_input2_smooth_filter)
mean_squared_error(original_discharge_output2_smooth_filter, LReg_prediction_charge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_input1_smooth_filter[:,1], (original_discharge_output1_smooth_filter.flatten() - original_discharge_input1_smooth_filter[:,0].flatten()),  marker='.', s=10, label='Train - Smooth Filtered')
ax.scatter(original_discharge_input1_filter[:,1], (original_discharge_output1_filter.flatten() - original_discharge_input1_filter[:,0].flatten()),  marker='.', s=10, label='Train - Filtered')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_input2_smooth_filter[:,1], (original_discharge_output2_smooth_filter.flatten() - original_discharge_input2_smooth_filter[:,0].flatten()),  marker='.', s=10, label='Test - Smooth Filtered')
ax.scatter(original_discharge_input2_filter[:,1], (original_discharge_output2_filter.flatten() - original_discharge_input2_filter[:,0].flatten()),  marker='.', s=10, label='Test - Filtered')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_input1_smooth_filter[:,1], (original_discharge_output1_smooth_filter.flatten() - original_discharge_input1_smooth_filter[:,0].flatten()),  marker='.', s=10, label='Train - Smooth Filtered')
ax.scatter(original_discharge_input2_smooth_filter[:,1], (original_discharge_output2_smooth_filter.flatten() - original_discharge_input2_smooth_filter[:,0].flatten()),  marker='.', s=10, label='Test - Smooth Filtered')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_input1_filter[:,1], (original_discharge_output1_filter.flatten() - original_discharge_input1_filter[:,0].flatten()),  marker='.', s=10, label='Train - Filtered')
ax.scatter(original_discharge_input2_filter[:,1], (original_discharge_output2_filter.flatten() - original_discharge_input2_filter[:,0].flatten()),  marker='.', s=10, label='Test - Filtered')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

### Opposite Mode Trained Smooth Filtered Data

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, True, True)

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input, original_charge_training_output, _ = datapreprocessing(scaled_data_2019_10_08_run1, False, True)

In [ ]:
combined_dataset = np.vstack((original_charge_training_input, original_discharge_training_input))
combined_to_ts_meter =  np.vstack((original_charge_training_output, original_discharge_training_output))

In [ ]:
plt_mean_squared_error = []
regression_model = LinearRegression()
regression_model.fit(original_discharge_training_input, original_discharge_training_output)

In [ ]:
LReg_prediction_charge_mode = regression_model.predict(original_charge_training_input)
mean_squared_error(original_charge_training_output, LReg_prediction_charge_mode, squared=False)

In [ ]:
plt_mean_squared_error = []
regression_model = LinearRegression()
regression_model.fit(original_charge_training_input, original_charge_training_output)

In [ ]:
LReg_prediction_charge_mode = regression_model.predict(original_discharge_training_input)
mean_squared_error(original_discharge_training_output, LReg_prediction_charge_mode, squared=False)

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, False)


fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0]), marker='.', s=10)
ax.set_xlabel("Load [kWh]")
ax.set_ylabel("Δ Inventory [%]")

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False)



fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()),  marker='.', s=10)
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1[:70]

In [ ]:
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
scaled_data_2019_10_22_run1

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, False, True)

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter((to_ts_meter.flatten() - from_ts_meter.flatten()), load, marker='.', s=10, label='Integrated Load Data')
ax.set_title("Discharge Mode Load Data", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Load [kWh]")
ax.set_xlabel("Δ Inventory [%]")

## Gaussian Process

### Discharge Mode

In [ ]:
k = gpflow.kernels.Linear([1., 1.])

In [ ]:
print_summary(k)

In [ ]:
m = gpflow.models.GPR(data=(original_discharge_training_input, original_discharge_training_output), kernel=k, mean_function=None)

In [ ]:
print_summary(m)

In [ ]:
opt = gpflow.optimizers.Scipy()

In [ ]:
opt_logs = opt.minimize(
    m.training_loss, m.trainable_variables, options=dict(maxiter=1000)
)
print_summary(m)

In [ ]:
GP_prediction_mean_discharge_mode_training, GP_prediction_variance_discharge_mode_training = m.predict_f(dataset_ts_meter)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(GP_prediction_mean_discharge_mode_training.numpy().flatten(), to_ts_meter, squared=False)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, False)

In [ ]:
GP_prediction_mean_discharge_mode, GP_prediction_variance_discharge_mode = m.predict_f(original_discharge_validation_input)

saver = gpflow.saver.Saver()
saver.save('/tmp/model', m)

tf.saved_model.save(m, './model')

load_model = tf.saved_model.load('./model')

print_summary(load_model)

GP_prediction_mean_discharge_mode

GP_prediction_variance_discharge_mode

In [ ]:
x1 = GP_prediction_mean_discharge_mode.numpy().flatten()

np.sqrt(np.mean((x1-to_ts_meter)**2))

(GP_prediction_mean_discharge_mode - GP_prediction_variance_discharge_mode).shape

sortedArr

sortedArr[0].shape

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(x1, original_discharge_validation_output, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, GP_prediction_mean_discharge_mode.numpy().flatten(), marker='.', s=10, label='Prediction', color='red')
ax.plot(to_ts_meter, to_ts_meter, label='Perfect')
ax.set_title("Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (x1 - original_discharge_validation_input[:,0]), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0]), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Load [kWh]")
ax.set_ylabel("Δ Inventory [%]")

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input_smooth_filter, original_discharge_training_output_filter, _ = datapreprocessing(scaled_data_2018_05_17_run2, True, True)

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input_smooth, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, True, True)

In [ ]:
import scipy

In [ ]:
distribution1 = [np.mean()]

In [ ]:
scipy.special.rel_entr(original_discharge_training_input_smooth_filter, original_discharge_validation_input_smooth)

In [ ]:
scipy.spatial.distance.jensenshannon(original_discharge_training_input_smooth_filter, original_discharge_validation_input_smooth)

#### Smooth Discharge mode data

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input_smooth, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(smooth_ts_f))],smooth_ts_f.flatten(), marker='o', s=10, label='Prediction', color='red')
ax.scatter( [x for x in range(0, len(ts_f))], ts_f.flatten(), marker='.', s=10, label='Prediction', color='blue')

In [ ]:
k = gpflow.kernels.Linear([1., 1.])

In [ ]:
print_summary(k)

In [ ]:
m = gpflow.models.GPR(data=(original_discharge_training_input_smooth, original_discharge_training_output), kernel=k, mean_function=None)

In [ ]:
print_summary(m)

In [ ]:
opt = gpflow.optimizers.Scipy()

In [ ]:
opt_logs = opt.minimize(
    m.training_loss, m.trainable_variables, options=dict(maxiter=1000)
)
print_summary(m)

In [ ]:
GP_prediction_mean_discharge_mode_training_smooth, GP_prediction_variance_discharge_mode_training_smooth = m.predict_f(dataset_ts_meter)

In [ ]:
mean_squared_error(GP_prediction_mean_discharge_mode_training_smooth.numpy().flatten(), to_ts_meter, squared=False)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input_smooth, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, True)

In [ ]:
GP_prediction_mean_discharge_mode, GP_prediction_variance_discharge_mode = m.predict_f(original_discharge_validation_input_smooth)

In [ ]:
x1 = GP_prediction_mean_discharge_mode.numpy().flatten()

np.sqrt(np.mean((x1-to_ts_meter)**2))

In [ ]:
original_discharge_validation_output.shape

In [ ]:
mean_squared_error(original_discharge_validation_output, GP_prediction_mean_discharge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, x1, marker='.', s=10, label='Prediction', color='red')
ax.plot(to_ts_meter, to_ts_meter, label='Perfect')
ax.set_title("Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input_smooth[:,1], (x1 - original_discharge_validation_input_smooth[:,0]), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input_smooth[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input_smooth[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

### Charging mode

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input, original_charge_training_output, _ = datapreprocessing(scaled_data_2019_10_08_run1, False)

In [ ]:
k = gpflow.kernels.Linear([1., 1.])

In [ ]:
print_summary(k)

In [ ]:
m = gpflow.models.GPR(data=(original_charge_training_input, original_charge_training_output), kernel=k, mean_function=None)

In [ ]:
print_summary(m)

In [ ]:
opt = gpflow.optimizers.Scipy()

opt = gpflow.train.AdamOptimizer(0.01)

In [ ]:
opt_logs = opt.minimize(
    m.training_loss, m.trainable_variables, options=dict(maxiter=1000)
)
print_summary(m)

In [ ]:
GP_prediction_mean_charge_mode_training, GP_prediction_variance_charge_mode_training = m.predict_f(dataset_ts_meter)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(GP_prediction_mean_charge_mode_training.numpy().flatten(), to_ts_meter, squared=False)

#### Validation

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False)

In [ ]:
GP_prediction_mean_charge_mode, GP_prediction_variance_charge_mode = m.predict_f(original_charge_validation_input)

tf.random.set_seed(1)  
samples = m.predict_f_samples(dataset_ts_meter, 10)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(load, (to_ts_meter - from_ts_meter), marker='.', s=10, label='Raw Data Integrated Load')
ax.set_title("Charge mode Raw Data", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Δ Inventory [%]")
ax.set_ylabel("Load [kWh]")

In [ ]:
mean_squared_error(original_charge_validation_output, GP_prediction_mean_charge_mode, squared=False)

In [ ]:
x1 = GP_prediction_mean_charge_mode.numpy().flatten()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (x1 - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()),  marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, GP_prediction_mean_charge_mode, marker='.', s=10, label='Prediction', color='red')
ax.plot(to_ts_meter, to_ts_meter, label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(load, (GP_prediction_mean_charge_mode), marker='.', s=10, label='Predicted', color='red')
ax.scatter(load, (to_ts_meter),  marker='.', s=10, label='Actual')
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Δ Inventory [%]")
ax.set_ylabel("Load [kWh]")

In [ ]:
instantaneous_load

In [ ]:
load_met = pd.Series(load.flatten())
final_ice_inventory = pd.Series(to_ts_meter.flatten())
initial_ice_inventory = pd.Series(from_ts_meter.flatten())
load_data = pd.DataFrame()
load_data["Initial [%]"] = initial_ice_inventory
load_data["Load [kWh]"] = load_met
load_data["Final [%]"] = final_ice_inventory

In [ ]:
load_data[:10]

## Discharge Mode - Interpolation

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, False)

In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
interp = LinearNDInterpolator(original_discharge_training_input, original_discharge_training_output)

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode = interp(dataset_ts_meter)
x1 = Interp__prediction_to_ts_meter_discharge_mode.flatten()
indices = np.argwhere(np.isnan(x1))
new_to_ts_meter = np.delete(to_ts_meter, indices)
new_x1 = np.delete(x1, indices)
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, False)

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode = interp(original_discharge_validation_input)

In [ ]:
x1 = Interp__prediction_to_ts_meter_discharge_mode.flatten()

### fewer points in RMSE than other models so mention this point

In [ ]:
np.count_nonzero(~np.isnan(x1))

In [ ]:
indices = np.argwhere(np.isnan(x1))

In [ ]:
new_to_ts_meter = np.delete(original_discharge_validation_output, indices)

In [ ]:
new_to_ts_meter.shape

In [ ]:
new_x1 = np.delete(x1, indices)

In [ ]:
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, Interp__prediction_to_ts_meter_discharge_mode, marker='.', s=10, label='Prediction', color='Red')
ax.set_title("Final Ice Inventory", fontdict={'fontsize': 10})
ax.plot(to_ts_meter, to_ts_meter, label='Perfect')
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode.flatten().shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (Interp__prediction_to_ts_meter_discharge_mode.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='Red')
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

#### Smooth Discharge mode data

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input_smooth, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, True)

In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
interp = LinearNDInterpolator(original_discharge_training_input_smooth, original_discharge_training_output)

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode = interp(dataset_ts_meter)
x1 = Interp__prediction_to_ts_meter_discharge_mode.flatten()
indices = np.argwhere(np.isnan(x1))
new_to_ts_meter = np.delete(to_ts_meter, indices)
new_x1 = np.delete(x1, indices)
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input_smooth, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, True)

In [ ]:
Interp__prediction_to_ts_meter_discharge_mode_smooth = interp(original_discharge_validation_input_smooth)

In [ ]:
x1 = Interp__prediction_to_ts_meter_discharge_mode_smooth.flatten()
indices = np.argwhere(np.isnan(x1))
new_to_ts_meter = np.delete(original_discharge_validation_output, indices)
new_x1 = np.delete(x1, indices)

In [ ]:
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, x1, marker='.', s=10, label='Prediction', color='Red')
ax.set_title("Final Ice Inventory", fontdict={'fontsize': 10})
ax.plot(to_ts_meter, to_ts_meter, label='Perfect')
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (x1 - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='Red')
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

## Charging Mode

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input, original_charge_training_output, _ = datapreprocessing(scaled_data_2019_10_08_run1, False)

In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
interp = LinearNDInterpolator(original_charge_training_input, original_charge_training_output)

In [ ]:
Interp__prediction_to_ts_meter_charge_mode = interp(from_ts_meter, load)
x1 = Interp__prediction_to_ts_meter_charge_mode.flatten()
indices = np.argwhere(np.isnan(x1))
new_to_ts_meter = np.delete(to_ts_meter, indices)
new_x1 = np.delete(x1, indices)
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

#### Validation

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False)

In [ ]:
Interp__prediction_to_ts_meter_charge_mode = interp(original_charge_validation_input)

In [ ]:
x1 = Interp__prediction_to_ts_meter_charge_mode.flatten()
indices = np.argwhere(np.isnan(x1))
new_to_ts_meter = np.delete(original_charge_validation_output, indices)
new_x1 = np.delete(x1, indices)

In [ ]:
mean_squared_error(new_to_ts_meter, new_x1, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter.flatten(), Interp__prediction_to_ts_meter_charge_mode.flatten(), marker='.', s=10, label='Prediction', color='Red')
ax.plot(to_ts_meter.flatten(), to_ts_meter.flatten(), label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (Interp__prediction_to_ts_meter_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

## Linear Regression - Discharge Mode

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, False)

In [ ]:
regression_model = LinearRegression()
regression_model.fit(original_discharge_training_input, original_discharge_training_output)

In [ ]:
regression_model.coef_

In [ ]:
regression_model.intercept_

In [ ]:
numpy.mean()

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, False)

In [ ]:
nonsmooth_load_validation = load

In [ ]:
LReg_prediction_discharge_mode_nonsmooth = regression_model.predict(original_discharge_validation_input)
mean_squared_error(original_discharge_validation_output, LReg_prediction_discharge_mode_nonsmooth, squared=False)

In [ ]:
nonsmooth_residuals = np.sqrt((LReg_prediction_discharge_mode_nonsmooth-to_ts_meter)**2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (LReg_prediction_discharge_mode_nonsmooth.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

from mpl_toolkits import mplot3d
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

!pip install ipympl

fig = plt.figure()
ax = plt.axes(projection='3d')
#ax = Axes3D(fig)
fig.set_dpi(450)

# Data for three-dimensional scattered points
zdata = LReg_prediction_discharge_mode_nonsmooth
xdata = from_ts_meter
ydata = load
ax.scatter3D(xdata, ydata, to_ts_meter, c=to_ts_meter, cmap='Greens');
ax.scatter3D(xdata, ydata, zdata, c=ydata, cmap='Reds');
plt.savefig('model.png', dpi=450)

In [ ]:
regression_model.coef_

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter, LReg_prediction_discharge_mode, marker='.', s=10, label='Predicted', color='Red')
ax.set_title("Actual vs Predicted Final Ice Inventory", fontdict={'fontsize': 10})
ax.plot(to_ts_meter, to_ts_meter, label='Perfect Fit')
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(load.flatten(), (from_ts_meter.flatten() - LReg_prediction_discharge_mode.flatten()), marker='.', s=10, label='Predicted', color='Red')
ax.scatter(load.flatten(), (from_ts_meter.flatten() - to_ts_meter.flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Δ Inventory [%]")
ax.set_ylabel("Load [kWh]")

In [ ]:
x5 = LReg_prediction_discharge_mode.flatten()
predictions = np.vstack((x5, to_ts_meter.flatten()))
sortedArr = predictions [ :, predictions[1].argsort()]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
plt.scatter([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0],  label='Linear Regression', marker='.')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
#plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.scatter([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1],  label='Actual Data', marker='.')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

#### Smooth discharge mode 

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input_smooth, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, True)

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(instantaneous_load_nonsmooth))],instantaneous_load_nonsmooth.flatten(), marker='o', s=10, label='Nonsmooth', color='red')
ax.scatter( [x for x in range(0, len(instantaneous_load_smooth))], instantaneous_load_smooth.flatten(), marker='.', s=10, label='Smooth', color='blue')
ax.legend()

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(smooth_training_load))],smooth_training_load.flatten(), marker='o', s=10, label='Smooth', color='red')
ax.scatter( [x for x in range(0, len(nonsmooth_training_load))], nonsmooth_training_load.flatten(), marker='.', s=10, label='Nonsmooth', color='blue')
ax.legend()

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(smooth_validation_load))],smooth_validation_load.flatten(), marker='o', s=10, label='Smooth', color='red')
ax.scatter( [x for x in range(0, len(nonsmooth_load_validation))], nonsmooth_load_validation.flatten(), marker='.', s=10, label='Nonsmooth', color='blue')
ax.legend()

smooth_training_load
smooth_validation_load
nonsmooth_load_validation
nonsmooth_training_load

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(smooth_ts_f))],smooth_ts_f.flatten(), marker='o', s=10, label='Prediction', color='red')
ax.scatter( [x for x in range(0, len(ts_f))], ts_f.flatten(), marker='.', s=10, label='Prediction', color='blue')

In [ ]:
plt_mean_squared_error = []
regression_model = LinearRegression()
regression_model.fit(original_discharge_training_input_smooth, original_discharge_training_output)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input_smooth, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, True)

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(instantaneous_load_nonsmooth))],instantaneous_load_nonsmooth.flatten(), marker='o', s=10, label='Nonsmooth', color='red')
ax.scatter( [x for x in range(0, len(instantaneous_load_smooth))], instantaneous_load_smooth.flatten(), marker='.', s=10, label='Smooth', color='blue')
ax.legend()

fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(smooth_ts_f))],smooth_ts_f.flatten(), marker='o', s=10, label='Prediction', color='red')
ax.scatter( [x for x in range(0, len(ts_f))], ts_f.flatten(), marker='.', s=10, label='Prediction', color='blue')

In [ ]:
LReg_prediction_discharge_mode_smooth = regression_model.predict(original_discharge_validation_input_smooth)
mean_squared_error(original_discharge_validation_output, LReg_prediction_discharge_mode_smooth, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input_smooth[:,1], (LReg_prediction_discharge_mode_smooth.flatten() - original_discharge_validation_input_smooth[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input_smooth[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input_smooth[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter( [x for x in range(0, len(LReg_prediction_discharge_mode_smooth))],LReg_prediction_discharge_mode_smooth.flatten(), marker='o', s=2,  label='Smooth', color='red')
ax.scatter( [x for x in range(0, len(LReg_prediction_discharge_mode_nonsmooth))], LReg_prediction_discharge_mode_nonsmooth.flatten(), marker='*', s=2, label='Nonsmooth', color='blue')
ax.scatter( [x for x in range(0, len(to_ts_meter))], to_ts_meter.flatten(), marker='.', s=4, label='Actual Data', color='Green')
ax.legend()

In [ ]:
nonsmooth_residuals.shape

In [ ]:
regression_model.coef_

In [ ]:
x1 = LReg_prediction_discharge_mode.flatten()

In [ ]:
np.sqrt(np.mean((x1-to_ts_meter)**2))

In [ ]:
to_ts_meter.shape

In [ ]:
LReg_prediction_discharge_mode_nonsmooth
LReg_prediction_discharge_mode_smooth

In [ ]:
x5 = LReg_prediction_discharge_mode.flatten()
predictions = np.vstack((LReg_prediction_discharge_mode_nonsmooth.flatten(), LReg_prediction_discharge_mode_smooth.flatten(), to_ts_meter.flatten()))
sortedArr = predictions [ :, predictions[2].argsort()]

In [ ]:
sortedArr

In [ ]:
sortedArr

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
plt.scatter([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0],  label='Linear Regression Nonsmooth', marker='.')
plt.scatter([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1],  label='Linear Regression Smooth', marker='.')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
#plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.scatter([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2],  label='Actual Data', marker='.')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

## Linear Regression - Charge Mode

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input, original_charge_training_output, _ = datapreprocessing(scaled_data_2019_10_08_run1, False)

In [ ]:
plt_mean_squared_error = []
regression_model = LinearRegression()
regression_model.fit(original_charge_training_input, original_charge_training_output)

In [ ]:
regression_model.coef_

In [ ]:
regression_model.intercept_

In [ ]:
LReg_prediction_charge_mode = regression_model.predict(dataset_ts_meter)
mean_squared_error(to_ts_meter, LReg_prediction_charge_mode, squared=False)

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False)

In [ ]:
LReg_prediction_charge_mode = regression_model.predict(original_charge_validation_input)
mean_squared_error(original_charge_validation_output, LReg_prediction_charge_mode, squared=False)

In [ ]:
regression_model.intercept_

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (LReg_prediction_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter.flatten(), LReg_prediction_charge_mode.flatten(), marker='.', s=10, label='Prediction', color='Red')
ax.plot(to_ts_meter.flatten(), to_ts_meter.flatten(), label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

"""X = np.linspace(min(from_ts_meter), max(from_ts_meter))
Y = np.linspace(min(load), max(load))
X, Y = np.meshgrid(X, Y)
plt.pcolormesh(X, Y, Z, shading='auto')"""
plt.plot(x, y, "ok", label="input point")
plt.plot(x, z, "ok", label="input point")
plt.legend()
plt.colorbar()
plt.axis("equal")
plt.show()

In [ ]:
interp

## Neural Network - Discharge Mode

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, False)

In [ ]:
model = Sequential()
model.add(Dense(15, activation = 'relu', input_shape = (2,), kernel_initializer='glorot_uniform'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))
model.compile(optimizer='adam', loss='mse')


In [ ]:
model.fit(original_discharge_training_input, original_discharge_training_output, epochs=300)

In [ ]:
NeuralNet_prediction_discharge_mode = model.predict(original_discharge_training_input)
mean_squared_error(original_discharge_training_output, NeuralNet_prediction_discharge_mode, squared=False)

### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, False)

In [ ]:
model.evaluate(original_discharge_validation_input, original_discharge_validation_output)
NeuralNet_prediction_discharge_mode = model.predict(original_discharge_validation_input)
mean_squared_error(original_discharge_validation_output, NeuralNet_prediction_discharge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter.flatten(), NeuralNet_prediction_discharge_mode.flatten(), marker='.', s=10, label='Prediction', color='red')
ax.plot(to_ts_meter.flatten(), to_ts_meter.flatten(), label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input[:,1], (NeuralNet_prediction_discharge_mode.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

#### Smooth discharge mode 

In [ ]:
scaled_data_2018_05_17_run2 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2018_05_17_run2 = scaled_data_2018_05_17_run2.resample('10T').mean()
original_discharge_training_input_smooth, original_discharge_training_output, _ = datapreprocessing(scaled_data_2018_05_17_run2, True)

In [ ]:
model = Sequential()
model.add(Dense(15, activation = 'relu', input_shape = (2,), kernel_initializer='glorot_uniform'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))
model.compile(optimizer='adam', loss='mse')
model.fit(original_discharge_training_input_smooth, original_discharge_training_output, epochs=300, verbose=0)

NeuralNet_prediction_discharge_mode = model.predict(dataset_ts_meter)
mean_squared_error(to_ts_meter, NeuralNet_prediction_discharge_mode, squared=False)

#### Validation

In [ ]:
scaled_data_2018_05_09_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2018_05_09_run1 = scaled_data_2018_05_09_run1.resample('10T').mean()
original_discharge_validation_input_smooth, original_discharge_validation_output, _ = datapreprocessing(scaled_data_2018_05_09_run1, True)

In [ ]:
model.evaluate(original_discharge_validation_input_smooth, original_discharge_validation_output)
NeuralNet_prediction_discharge_mode = model.predict(original_discharge_validation_input_smooth)
mean_squared_error(original_discharge_validation_output, NeuralNet_prediction_discharge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter.flatten(), NeuralNet_prediction_discharge_mode.flatten(), marker='.', s=10, label='Prediction', color='red')
ax.plot(to_ts_meter.flatten(), to_ts_meter.flatten(), label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_discharge_validation_input_smooth[:,1], (NeuralNet_prediction_discharge_mode.flatten() - original_discharge_validation_input_smooth[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_discharge_validation_input_smooth[:,1], (original_discharge_validation_output.flatten() - original_discharge_validation_input_smooth[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

## Neural Network - Charge Mode

In [ ]:
scaled_data_2019_10_08_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
#Q_data = scaled_data_2018_05_09_run1['ts_meter']
scaled_data_2019_10_08_run1 = scaled_data_2019_10_08_run1.resample('10T').mean()
original_charge_training_input, original_charge_training_output, _ = datapreprocessing(scaled_data_2019_10_08_run1, False)

In [ ]:
model = Sequential()
model.add(Dense(15, activation = 'relu', input_shape = (2,), kernel_initializer='glorot_uniform'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))
model.compile(optimizer='adam', loss='mse')
model.fit(original_charge_training_input, original_charge_training_output, epochs=300, verbose=0)

### Validation

In [ ]:
scaled_data_2019_10_22_run1 = data_files_fromS3('.PATH/sHeader.csv', '.PATH/scaledData.csv', "S3 BUCKET")
scaled_data_2019_10_22_run1 = scaled_data_2019_10_22_run1.resample('10T').mean()
original_charge_validation_input, original_charge_validation_output, _ = datapreprocessing(scaled_data_2019_10_22_run1, False)

In [ ]:
model.evaluate(original_charge_validation_input, original_charge_validation_output)
NeuralNet_prediction_charge_mode = model.predict(original_charge_validation_input)
mean_squared_error(original_charge_validation_output, NeuralNet_prediction_charge_mode, squared=False)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(to_ts_meter.flatten(), NeuralNet_prediction_charge_mode.flatten(), marker='.', s=10, label='Prediction', color='Red')
ax.plot(to_ts_meter.flatten(), to_ts_meter.flatten(), label='Perfect')
ax.set_title("Actual vs Prediction of Final Ice Inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_xlabel("Actual [%]")
ax.set_ylabel("Prediction [%]")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_dpi(450)
ax.scatter(original_charge_validation_input[:,1], (NeuralNet_prediction_charge_mode.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Prediction', color='red')
ax.scatter(original_charge_validation_input[:,1], (original_charge_validation_output.flatten() - original_charge_validation_input[:,0].flatten()), marker='.', s=10, label='Actual')
ax.set_title("Load vs Change in ice inventory", fontdict={'fontsize': 10})
ax.legend()
ax.legend(loc=2, prop={'size': 6})
ax.set_ylabel("Δ Inventory [%]")
ax.set_xlabel("Load [kWh]")

### Discharge Mode All models combined plot

In [ ]:
GP_prediction_mean_discharge_mode, GP_prediction_mean_charge_mode, GP_prediction_variance_discharge_mode, GP_prediction_variance_charge_mode
Interp__prediction_to_ts_meter_discharge_mode, Interp__prediction_to_ts_meter_charge_mode
LReg_prediction_discharge_mode, LReg_prediction_charge_mode
NeuralNet_prediction_discharge_mode, NeuralNet_prediction_charge_mode

In [ ]:
GP_prediction_mean_discharge_mode - LReg_prediction_discharge_mode

In [ ]:
plt.plot([x for x in range(0, len(GP_prediction_mean_discharge_mode))], GP_prediction_mean_discharge_mode, linewidth=3)
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
plt.plot([x for x in range(0, len(Interp__prediction_to_ts_meter_discharge_mode))], Interp__prediction_to_ts_meter_discharge_mode, linewidth=2)
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
plt.plot([x for x in range(0, len(LReg_prediction_discharge_mode))], LReg_prediction_discharge_mode, linewidth=1)
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
plt.plot([x for x in range(0, len(NeuralNet_prediction_discharge_mode))], NeuralNet_prediction_discharge_mode, linewidth=0.5)
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
plt.rcParams['figure.dpi'] = 3000
plt.show()

In [ ]:
GP_prediction_mean_discharge_mode

In [ ]:
plt.rcParams['figure.dpi'] = 600

In [ ]:
to_ts_sorted = to_ts_meter.flatten()
to_ts_sorted.sort()

In [ ]:
x1 = GP_prediction_mean_discharge_mode.numpy().flatten()
x2 = GP_prediction_mean_charge_mode.numpy().flatten()
x3 = Interp__prediction_to_ts_meter_discharge_mode.flatten()
x4 = Interp__prediction_to_ts_meter_charge_mode.flatten()
x5 = LReg_prediction_discharge_mode.flatten()
x6 = LReg_prediction_charge_mode.flatten()
x7 = NeuralNet_prediction_discharge_mode.flatten()
x8 = NeuralNet_prediction_charge_mode.flatten()
predictions = np.vstack((x1, x3, x5, x7, to_ts_meter.flatten()))
sortedArr = predictions [ :, predictions[4].argsort()]

plt.plot([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0], linewidth=0.5, label='GP Prediction')
#plt.plot([x for x in range(0, len(to_ts_sorted))], to_ts_sorted, linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
#plt.plot([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1], linewidth=0.5, label='Interpolation')
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
#plt.plot([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2], linewidth=0.5, label='Linear Regression')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
#plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.plot([x for x in range(0, sortedArr[4].shape[0])], sortedArr[4], linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

In [ ]:
#plt.plot([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0], linewidth=0.5, label='GP Prediction')
#plt.plot([x for x in range(0, len(to_ts_sorted))], to_ts_sorted, linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
plt.plot([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1], linewidth=0.5, label='Interpolation')
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
#plt.plot([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2], linewidth=0.5, label='Linear Regression')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
#plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.plot([x for x in range(0, sortedArr[4].shape[0])], sortedArr[4], linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

In [ ]:
#plt.plot([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0], linewidth=0.5, label='GP Prediction')
#plt.plot([x for x in range(0, len(to_ts_sorted))], to_ts_sorted, linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
#plt.plot([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1], linewidth=0.5, label='Interpolation')
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
plt.plot([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2], linewidth=0.5, label='Linear Regression')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
#plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.plot([x for x in range(0, sortedArr[4].shape[0])], sortedArr[4], linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

In [ ]:
#plt.plot([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0], linewidth=0.5, label='GP Prediction')
#plt.plot([x for x in range(0, len(to_ts_sorted))], to_ts_sorted, linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
#plt.plot([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1], linewidth=0.5, label='Interpolation')
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
#plt.plot([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2], linewidth=0.5, label='Linear Regression')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.plot([x for x in range(0, sortedArr[4].shape[0])], sortedArr[4], linewidth=0.5, label='Actual Data')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()

In [ ]:
plt.plot([x for x in range(0, sortedArr[0].shape[0])], sortedArr[0], linewidth=0.5, label='GP Prediction')
#plt.plot([x for x in range(0, len(to_ts_sorted))], to_ts_sorted, linewidth=1, label='Actual Data')
#plt.scatter([x for x in range(0, len(GP_prediction_mean_charge_mode))], GP_prediction_mean_charge_mode)
plt.plot([x for x in range(0, sortedArr[1].shape[0])], sortedArr[1], linewidth=0.5, label='Interpolation')
#plt.scatter([x for x in range(0, len(Interp__prediction_to_ts_meter_charge_mode))], Interp__prediction_to_ts_meter_charge_mode)
plt.plot([x for x in range(0, sortedArr[2].shape[0])], sortedArr[2], linewidth=0.5, label='Linear Regression')
#plt.scatter([x for x in range(0, len(LReg_prediction_charge_mode))], LReg_prediction_charge_mode)
plt.plot([x for x in range(0, sortedArr[3].shape[0])], sortedArr[3], linewidth=0.5, label='Neural Net')
plt.plot([x for x in range(0, sortedArr[4].shape[0])], sortedArr[4], linewidth=0.5, label='Actual Data')
#plt.scatter([x for x in range(0, len(NeuralNet_prediction_charge_mode))], NeuralNet_prediction_charge_mode)
#plt.set
plt.legend()
plt.show()